In [1]:
import pandas as pd
import numpy as np
import os
import csv

cwd = os.getcwd()


In [125]:
!cat combined_full_data.csv | python create_training_data.py > training_data.txt

In [126]:
!head training_data.txt

id|text|in_reply_to_screen_name|in_reply_to_user_id|in_reply_to_status_id|sarcastic
977818307960197120|RT @betriumBets: Betrium as featured on ICO Alert.https://t.co/efjXFxfNLl#Betrium #Tokensale #ICO #Bitcoin #Ethereum #TheFutureOfGambli…|None|None|None|0
977781153804496896|PCA Trainee-Willcox, AZ, #Willcox, #AZ, #AgricultureForestryFishing https://t.co/GUVdzEyfi8 #job #hiring|None|None|None|0
977782880297418752|RT @wefundanydeal: WOW WOW WOW ONLY 2 days to go till our Pre Sale!!  Be part of this Financial Evolution. Do not miss out on our bonus dis…|None|None|None|0
977820339215335430|☮️🐱#LateNight #Party  another song blasting while on the freeway... https://t.co/wLRyVZkpKv?|None|None|None|0
977804716615729152|RT @spotthedealusa: $59.94 (Reg $69.99) #Amazon Today Perfect #Kids #Easter Gifts #LOLSurprise Big (aff)  https://t.co/fUKcn6S9M2 https://…|None|None|None|0
977810260848242688|RT @ampomata: This is my painting Kids Collecting Marine Shells. You can check it out here: https://t

In [127]:
#df_sarcastic = pd.read_csv('merged_data_v1_1.csv', sep = '|')

df = pd.DataFrame()



with open('training_data.txt', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            header= row
            df = pd.DataFrame(columns = row)
        else:
            if len(header) != len(row):
                continue
            else:
                addition = {i:j for i, j in zip(header,row)}
                df = df.append(addition, ignore_index=True )
                


df.tail()        

,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic
3768,977761572520067072,@FLAVOLS @robbysoave History? It was made by ...,FLAVOLS,99869658,977752228936241152,1
3769,977761845766324226,Yaoi desu ne? this development in the near end...,None,None,None,1
3770,977762632173215745,Scrolling through Facebook when suddenly... I ...,None,None,None,1
3771,977762993244024833,I cant wait to start my fully semi-automatic A...,None,None,None,1
3772,977763216800526336,Can someone make me cum on kik? Add me : REMUD...,None,None,None,1


In [128]:
for i in np.unique(df['in_reply_to_status_id']):
    if len(i) != 18:
        print (i)

None


In [129]:
#### true exmaples : 
### false examples :
import re
status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_storage = []

for j in df[df['in_reply_to_status_id'] != 'None'].index:
    if (((df['in_reply_to_screen_name'][j][0] != '#')) & 
        bool(re.match(status_id_pattern,df['in_reply_to_status_id'][j]))):
        index_storage.append(j)
    else: continue

len(index_storage)

678

In [130]:
#want the clean rows that do not have retreats

status_id_pattern = r'^[0-9]{18}$'
response_name_not_pattern = r'^#'


index_alt = []

for j in df[df['in_reply_to_status_id'] == 'None'].index:
    if (bool(re.match(status_id_pattern,df['id'][j]))):
        index_alt.append(j)
    else: continue

len(index_alt)


3095

In [131]:
#subset those data sets that will not have as much trouble

sub_df = df.iloc[index_storage]
sub_df['url']= "https://twitter.com/"+sub_df['in_reply_to_screen_name']+"/status/"+sub_df['in_reply_to_status_id']
#max(index_storage)
sub_df.head()

/Users/saulgrimaldo1/anaconda/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url
35,977781585490493440,@CAoutcast @leononleonard @PaulGMcC1967 @BookT...,CAoutcast,3309854312,977780736676646912,0,https://twitter.com/CAoutcast/status/977780736...
96,977780707950022656,@Topdekfireblast @BillGuindon Guess that makes...,Topdekfireblast,965900497457680384,977691551802576897,0,https://twitter.com/Topdekfireblast/status/977...
142,977783721343442944,#NewsAlert!!: #MohaveValleyAZ: #MohaveCountyDe...,ZachNews,107184476,977783719057489920,0,https://twitter.com/ZachNews/status/9777837190...
295,977817667322118149,@EdKrassen I just keep fighting the urge for m...,EdKrassen,132339474,977440125012578305,0,https://twitter.com/EdKrassen/status/977440125...
314,977781563957096448,.@ClareMeans on #Periscope: #Singing on #santa...,ClareMeans,66454154,977780762849067010,0,https://twitter.com/ClareMeans/status/97778076...


In [132]:
#want the clean rows that do not have retreats
!conda install beautifulsoup


Fetching package metadata ...........

PackageNotFoundError: Packages missing in current channels:
            
  - beautifulsoup

We have searched for the packages in the following channels:
            
  - https://repo.continuum.io/pkgs/main/osx-64
  - https://repo.continuum.io/pkgs/main/noarch
  - https://repo.continuum.io/pkgs/free/osx-64
  - https://repo.continuum.io/pkgs/free/noarch
  - https://repo.continuum.io/pkgs/r/osx-64
  - https://repo.continuum.io/pkgs/r/noarch
  - https://repo.continuum.io/pkgs/pro/osx-64
  - https://repo.continuum.io/pkgs/pro/noarch
            



In [133]:
sub_df

,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url
35,977781585490493440,@CAoutcast @leononleonard @PaulGMcC1967 @BookT...,CAoutcast,3309854312,977780736676646912,0,https://twitter.com/CAoutcast/status/977780736...
96,977780707950022656,@Topdekfireblast @BillGuindon Guess that makes...,Topdekfireblast,965900497457680384,977691551802576897,0,https://twitter.com/Topdekfireblast/status/977...
142,977783721343442944,#NewsAlert!!: #MohaveValleyAZ: #MohaveCountyDe...,ZachNews,107184476,977783719057489920,0,https://twitter.com/ZachNews/status/9777837190...
295,977817667322118149,@EdKrassen I just keep fighting the urge for m...,EdKrassen,132339474,977440125012578305,0,https://twitter.com/EdKrassen/status/977440125...
314,977781563957096448,.@ClareMeans on #Periscope: #Singing on #santa...,ClareMeans,66454154,977780762849067010,0,https://twitter.com/ClareMeans/status/97778076...
418,977820097912832001,@Michael2014abc @Kahil @DanDuke2013 @WestsideF...,Michael2014abc,2411474370,977819365360926720,0,https://twitter.com/Michael2014abc/status/9778...
451,977779830472110080,@richbrian You got that asian persuasion. That...,richbrian,178690996,977777946088718336,0,https://twitter.com/richbrian/status/977777946...
489,977779887527272448,@ParisAMDParis https://t.co/9amI3T8qV4Graphic ...,ParisAMDParis,875280010877296640,977517492389253120,0,https://twitter.com/ParisAMDParis/status/97751...
562,977810879948390400,@CricketAus When the Ashes 2017/18 story is wr...,CricketAus,17692554,977802048627331072,0,https://twitter.com/CricketAus/status/97780204...
611,977818059376340992,"@popcornshed Honest, loyal, loving. #Competiti...",popcornshed,3139923819,977809861894340608,0,https://twitter.com/popcornshed/status/9778098...


In [134]:
from bs4 import BeautifulSoup
import requests
import re
import json
def preceding_tweet(URL_):

    ### TODO  URL_ will be a function of the table 
    #URL_ = "https://twitter.com/verobchfinfan/status/976155584448868353"
    page = requests.get(URL_)

    soup = BeautifulSoup(page.content, 'html.parser')

    html = list(soup.children)[2]

    body = list(html.children)[3]


    ###  entity in tweet that gives a full tweet with low confidence in structure

    p1 = list(body.children)[61]

    p1a = list(p1.children)[5]
    p1b=list(list(p1a.children)[3].children)[1]
    set_of_parent = set(list(p1b.children)[1].get_text().split('\n'))
    set_of_parent = list(map(lambda x: x.strip(),set_of_parent))

   
    ###  entity in tweet that gives a partial tweet with high confidence in structure
    p2 = list(body.children)[65]

    p2a= json.loads(p2.__dict__['attrs']['value'])

    p2a['initialState']['title']

    pattern_1 = r'^[A-Za-z0-9 ]* on Twitter: "'
    pattern_2 = '\u2026'

    if re.match(pattern_1, p2a['initialState']['title']):
        start_index = len(re.match(pattern_1, p2a['initialState']['title']).group(0))
    else:
        start_index = None
    
    
    if pattern_2 in p2a['initialState']['title']:
        stop_index = p2a['initialState']['title'].rfind(pattern_2)
    else: 
        stop_index = None

    partial_tweet = p2a['initialState']['title'][start_index:stop_index]
    
    
    #### pull out tweet preceding

    potential_tweet_list = [el for el in set_of_parent if partial_tweet in el]

    len_counter = 0

    for i in potential_tweet_list:
        current_len = len(i)
        if current_len> len_counter:
            preceding_tweet_x = i
        else:
            pass
    if len(potential_tweet_list)==0:
        return partial_tweet
    else:
        return preceding_tweet_x

In [135]:
storage_pretweet = []

for x in sub_df['url']:
    url = x.strip()
    try:
        storage_pretweet.append(preceding_tweet(url))
    except:
        storage_pretweet.append(None)
    

In [136]:
storage_pretweet

['Suzi S. on Twitter: "Okay.. it was a Clinton - Bush - Obama offsh**t....Oh well, all the lax law liberals and their Globalist Marxist agenda.. ho hum...... Time to reinstate the death penalty!',
 'Jesse Nelson 🇺🇸🇨🇦🇦🇺🇸🇪🇩🇰🇳🇴🇬🇧 on Twitter: "In Utah the NRA terrorists showed up with a machine gun mounted on a truck to intimidate children',
 '#NewsAlert!!: #MohaveValleyAZ: #MohaveCountyDepartmentofPublicHealth #Issues #Public #Health #Notice regarding #Egg #Drop #Event at #MohaveValleyCommunityPark. https://t.co/RXDmJlIz2f #MohavaCountyDofPH #EggDropandFamilyFun2018 #MVFD #MohaveValleyFD #MohaveValleyFire"',
 'Ed Krassenstein 💎 on Twitter: "What is it with Trump sleeping with young beautiful women and then comparing them to his own daughter, Ivanka?  Is anyone else at least a bit disturbed about this, or is it perfectly normal to constantly compare your lovers to your own children?"',
 '#Singing on #santamonicapier ❤️🎸 #Music #acoustic @jimmyfallon https://t.co/6MGEdXgMKp"',
 'Unfortunate

In [137]:
sub_df['first_preceding_tweet'] = storage_pretweet

sub_df.head()

/Users/saulgrimaldo1/anaconda/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,text,in_reply_to_screen_name,in_reply_to_user_id,in_reply_to_status_id,sarcastic,url,first_preceding_tweet
35,977781585490493440,@CAoutcast @leononleonard @PaulGMcC1967 @BookT...,CAoutcast,3309854312,977780736676646912,0,https://twitter.com/CAoutcast/status/977780736...,"Suzi S. on Twitter: ""Okay.. it was a Clinton -..."
96,977780707950022656,@Topdekfireblast @BillGuindon Guess that makes...,Topdekfireblast,965900497457680384,977691551802576897,0,https://twitter.com/Topdekfireblast/status/977...,"Jesse Nelson 🇺🇸🇨🇦🇦🇺🇸🇪🇩🇰🇳🇴🇬🇧 on Twitter: ""In Ut..."
142,977783721343442944,#NewsAlert!!: #MohaveValleyAZ: #MohaveCountyDe...,ZachNews,107184476,977783719057489920,0,https://twitter.com/ZachNews/status/9777837190...,#NewsAlert!!: #MohaveValleyAZ: #MohaveCountyDe...
295,977817667322118149,@EdKrassen I just keep fighting the urge for m...,EdKrassen,132339474,977440125012578305,0,https://twitter.com/EdKrassen/status/977440125...,"Ed Krassenstein 💎 on Twitter: ""What is it with..."
314,977781563957096448,.@ClareMeans on #Periscope: #Singing on #santa...,ClareMeans,66454154,977780762849067010,0,https://twitter.com/ClareMeans/status/97778076...,#Singing on #santamonicapier ❤️🎸 #Music #acous...


In [155]:
index_alt

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187

In [ ]:
merged_data = pd.concat([df.iloc[index_alt],sub_df])

print(merged_data.shape)

merged_data.tail(10)

In [139]:
import re
def clean_text(text):
    text = re.sub(r"(\||\'|\"|\n|#sarcasm|#sarcastic)"," ", str(text),flags=re.IGNORECASE)
    return text

In [154]:
merged_data['first_preceding_tweet_clean'] = merged_data['first_preceding_tweet'].apply(clean_text)


merged_data_clean = merged_data[['text', 'first_preceding_tweet_clean','sarcastic']]

merged_data_clean.to_csv("merged_data_v2.csv", sep = '|', index = False)

In [150]:
merged_data_clean.head(10)

,text,first_preceding_tweet_clean,sarcastic
0,RT @FlamingoCandles: #FreebieFriday RT &amp; f...,nan,0
1,1.#aafaque_ehsens_ideamines_sites Explore our ...,nan,0
2,RT @lancechain: LanceChain is the first 0% com...,nan,0
3,RT @GocasePro: 🔥#Giveaway 12 Hours👉 Click http...,nan,0
4,BTC bullish scenario https://t.co/0DhLN4VAB4 ...,nan,0
5,PrimeHash - New #Bitcoin Mining!* 190 GHs on s...,nan,0
6,RT @relationshipsdw: #Blog 12: Why helicopter ...,nan,0
7,"Want to face your fears and be free to live, l...",nan,0
8,#Health just started trending with 22979 tweet...,nan,0
9,#vintage #Growing up in northern Kenya in the ...,nan,0


In [156]:
!tail merged_data_v2.csv

@donttrythis Dam there must be a tell of a lot of mentally ill people in the USA. I mean more than most 1st world countries. Must be something in the water or how they are  brought up. |You are wronghttps://twitter.com/tlorens/status/977254283354746886 …|1
@marcoPalba @CNNPolitics @CNN Yup because they look bad azz banning them wont do anything. |Most of us want a ban on ARs and that is it. Stricter background checks too.|1
@DanTheBeepMan @danabrams Shouldnt have beeped the man, dood. #livepd  #fulldisclosure #hesnotbuyingpizza|Dan the Beep Man 👮‍♂️🚔 on Twitter:  Don t think I have that much money!|1
@prrobbins My wife says the same thing to me.Wife: You need to adapt to me, not  expect me to you.Me: WTF is this world coming to. Lol.😂  😜 https://t.co/zSmCUpPe8z|Traders need to adapt to the market. Not expect the market to adapt to them. |1
@prrobbins Now if my wife &amp; girlfriend could just get along... Blahhh Lol.😂 😂 💩  #kidding 😜 https://t.co/YgQjztfivl|Happy wife. Happy life.|1
@c